In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [36]:
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import pipeline

In [3]:
mdf=pd.read_csv('/kaggle/input/fake-real-news/news_dataset.csv')

In [4]:
mdf.head()

,label,text
0,REAL,Payal has accused filmmaker Anurag Kashyap of ...
1,FAKE,A four-minute-long video of a woman criticisin...
2,FAKE,"Republic Poll, a fake Twitter account imitatin..."
3,REAL,"Delhi teen finds place on UN green list, turns..."
4,REAL,Delhi: A high-level meeting underway at reside...


In [5]:
p=mdf.label
p.value_counts(normalize=True)

label
FAKE    0.503352
REAL    0.496648
Name: proportion, dtype: float64

In [6]:
mdf.isnull().sum()

label    0
text     8
dtype: int64

In [7]:
df = mdf.dropna()

In [8]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [9]:
p=df.label
p.value_counts(normalize=True)

label
FAKE    0.502822
REAL    0.497178
Name: proportion, dtype: float64

In [12]:

df = df.replace({'FAKE': 0, 'REAL': 1})
df = df.rename(columns={'label': 'labels'})
dataset = Dataset.from_pandas(df)

In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [17]:
tokenized_df = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3721 [00:00<?, ? examples/s]

In [18]:
tokenized_df = tokenized_df.train_test_split(test_size=0.3)

In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
accuracy = evaluate.load("accuracy")

In [25]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [26]:
id2label = {0: "FAKE", 1: "REAL"}
label2id = {"FAKE": 0, "REAL": 1}

In [29]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
training_args = TrainingArguments(
    output_dir="FakeNewsBERT",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.02,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df['train'],
    eval_dataset=tokenized_df['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.008695,0.999105
2,No log,0.008559,0.999105


TrainOutput(global_step=326, training_loss=1.0183631682459928e-05, metrics={'train_runtime': 333.5402, 'train_samples_per_second': 15.614, 'train_steps_per_second': 0.977, 'total_flos': 1370282376314880.0, 'train_loss': 1.0183631682459928e-05, 'epoch': 2.0})

In [39]:
text = "An image of Brazilian President Jair Bolsonaro breaking down at a Thanksgiving service in Brasilia has resurfaced with claims misidentifying him as the heads of state of Italy, which has overtaken China as the new hotbed of the Coronavirus. The viral image attributes a fake quote to the Italian leaders as well. The quote throws light on the grim situation in Italy post the pandemic and the overburdened health infrastructure there. One such post reads, Italian Prime Minister Giuseppe Conte, We have lost control. We have killed the epidemic physically and mentally. Can't understand what more we can do, all solutions are exhausted on ground. Our only hope remains up in the sky, God rescue your people. Below is one such pot.The image was also posted on a meme page, Memes Radar, with the same narrative. The screenshot of the post is now viral on Whatsapp. The admin of the page later acknowledged in the comment that the photo is not of the Italian president Sergio Matarella. It is not clear whether the graphic began as a meme but BOOM came across multiple Facebook pages where netizens thought it to be real.Italy has been declared the new epicenter of COVID-19, with as many as 4825 deaths so far. 53,578 have been infected with the virus so far. The staggering numbers are the most for a single country till now. The country has been locked down for over 10 days post the outbreak."
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/FakeNewsBERT/checkpoint-326")
new_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

classifier = pipeline("text-classification", model=model, tokenizer=new_tokenizer)
classifier(text)

[{'label': 'FAKE', 'score': 0.9999977350234985}]